In [1]:
%matplotlib inline
# Import bokeh packages
from bokeh.layouts import column
from bokeh.models import CategoricalColorMapper, ColumnDataSource, CustomJS, Legend, Range, Range1d, Slider
from bokeh.palettes import Category20
from bokeh.plotting import figure, output_file, save, show

# Import python packages
from IPython.display import Image
#import graphviz
import numpy as np
#import pydotplus 
from sklearn import tree
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans

In [2]:
# Load iris data
iris = load_iris()
trainingData = iris.data
featureNames = iris.feature_names
trainingLabels = iris.target
labelNames = iris.target_names

# Bucket the training data points by training label (i.e. setosa, versicolor or virginica)
# This will make it easier to generate a legend for the plot
label_0 = []
label_1 = []
label_2 = []
for i in range(len(trainingLabels)):
    if trainingLabels[i] == 0:
        label_0.append(i)
    elif trainingLabels[i] == 1:
        label_1.append(i)
    else:
        label_2.append(i)

In [3]:
# Specify output html file 
#output_file("kmeans_model.html")

pca = PCA(n_components=2)
trainingData = pca.fit_transform(trainingData)
np.random.seed(1)
model = KMeans(n_clusters=3, n_init=1,init='random')
model.fit(trainingData)
labels = model.labels_

    
# Determine the limits of the plot
x_min = min([dataPoint[0] for dataPoint in trainingData]) - 1
x_max = max([dataPoint[0] for dataPoint in trainingData]) + 1
x_range = Range1d(x_min, x_max, bounds = (x_min, x_max))

y_min = min([dataPoint[1] for dataPoint in trainingData]) - 1
y_max = max([dataPoint[1] for dataPoint in trainingData]) + 1
y_range = Range1d(y_min, y_max, bounds = (y_min, y_max))

# Create a mesh grid based on the plot limits, then classify the mesh using the trained model
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.2), np.arange(y_min, y_max, 0.1))
z = model.predict(np.c_[xx.ravel(), yy.ravel()])
zz = z.reshape(xx.shape)
print "Changed"
# Create bokeh figure
bokeh_plot = figure(plot_width=500,
                    plot_height=500,
                    x_range = x_range,
                    y_range = y_range,
                    )

# Define color palettes for plots (light for decision surfaces, dark for data points)
light_palette = [Category20[6][2*i + 1] for i in range(3)]
dark_palette = [Category20[6][2*i] for i in range(3)]

# Plot the mesh grid on the bokeh figure as an image
# Note: "Z" here is the mesh grid predictions, and it must be contained in a list (i.e. "[Z]" not "Z")
bokeh_plot.image(image=[zz],
                 alpha = 0,
                 x=x_min,           
                 y=y_min,
                 dw=(x_max-x_min),
                 dh=(y_max-y_min),
                 palette = light_palette
                )



# Bucket the training data points by training label (i.e. setosa, versicolor or virginica)
# This will make it easier to generate a legend for the plot

label_0 = []
label_1 = []
label_2 = []
for i in range(len(labels)):
    if labels[i] == 0:
        label_0.append(i)
    elif labels[i] == 1:
        label_1.append(i)
    else:
        label_2.append(i)
        
# Plot data points in the label_0 bucket
bokeh_plot.circle([trainingData[i][0] for i in label_0], 
                  [trainingData[i][1] for i in label_0],
                  size = 4,
                  fill_color = dark_palette[0],
                  line_color = dark_palette[0],
                  legend = "Cluster One"
                 )

# Plot data points in the label_1 bucket
bokeh_plot.circle([trainingData[i][0] for i in label_1], 
                  [trainingData[i][1] for i in label_1],
                  size = 4,
                  fill_color = dark_palette[1],
                  line_color = dark_palette[1],
                  legend = "Cluster Two"
                 )

# Plot data points in the label_2 bucket
bokeh_plot.circle([trainingData[i][0] for i in label_2], 
                  [trainingData[i][1] for i in label_2],
                  size = 4,
                  fill_color = dark_palette[2],
                  line_color = dark_palette[2],
                  legend = "Cluster Three"
                 )


# Label axes, place legend
#bokeh_plot.xaxis.axis_label = featureNames[0]
#bokeh_plot.yaxis.axis_label = featureNames[1]
bokeh_plot.legend.location = "bottom_left"

# Show the resulting bokeh plot (this will save the html file and open it automatically)
# Alternatively, you could replace "show" with "save", which will save the file without opening it
show(bokeh_plot)

Changed


In [ ]:
'''
random_seed = Slider(title="RandomSeed", value=1.0, start=0, end=10)
#freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1)

# Set up callbacks
def update(attrname, old, new):
    # Get the current slider values
    seed = random_seed.value
    
    # Update the data for the new curve
    source.data = dict(x=x, y=a*sin(k*x))

amplitude.on_change('value', update)
freq.on_change('value', update)

# Set up layout and add to document
inputs = widgetbox(amplitude, freq)
curdoc().add_root(row(inputs, plot, width=1200))
'''


In [9]:
from bokeh.io import vplot
import pandas as pd
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.models.widgets import Button
from bokeh.plotting import figure, output_file, show

output_file("load_data_buttons.html")

#df1 = pd.read_csv("data_file_1.txt")
#df2 = pd.read_csv("data_file_2.txt")

plot = figure(plot_width=400, plot_height=400)

source = ColumnDataSource(data=dict(x=[0, 1], y=[0, 1]))
source2 = ColumnDataSource(data=dict(x=[1,2], y = [5,3]))

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source, source2=source2), code="""
        var data = source.get('data');
        var data2 = source2.get('data');
        data['x'] = data2['x' + cb_obj.get("name")];
        data['y'] = data2['y' + cb_obj.get("name")];
        source.trigger('change');
    """)

toggle1 = Button(label="Load data file 1", callback=callback, name="1")
toggle2 = Button(label="Load data file 2", callback=callback, name="2")

layout = vplot(toggle1, toggle2, plot)

show(layout)

INFO:bokeh.core.state:Session output file 'load_data_buttons.html' already exists, will be overwritten.
/Users/sarahkelley/anaconda/envs/tensorflow/lib/python2.7/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: bokeh.io.vplot() was deprecated in Bokeh 0.12.0 and will be removed, use bokeh.models.layouts.Column instead.
  warn(message)
